In [1]:
from txtai.embeddings import Embeddings
from IPython.display import display

/Users/jeffhara/PycharmProjects/similar_tags_service/venv/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [65]:
embeddings = Embeddings({"path": "sentence-transformers/all-mpnet-base-v2"})

In [3]:
import pandas as pd

post_df = pd.read_json("data/preprocessed_data.jsonl", orient="records")

In [5]:
embeddings.close()

In [63]:
from collections import Counter

tags = []
for i, row in post_df.iterrows():
    for tag in row["tags"] + row["root_tags"]:
        tags.append(tag.lower())

documents = []
tag_by_id = {}
for i, (tag, count) in enumerate(Counter(tags).most_common()):
    documents.append((i, tag, None))
    tag_by_id[i] = tag
len(documents)

278808

In [64]:
embeddings.index(documents)

KeyboardInterrupt: 

In [61]:
tag_df = pd.read_csv("data/Trial-supplement-tag-followers.csv")

for i, row in tag_df.tail(1000).sample(5).iterrows():
    print('seed:', row['tag'])
    for doc, score in embeddings.search(row['tag'], 10):
        if 0.5 < score < 0.9:
            print(tag_by_id[doc], score)
    print()

seed: tuserkaty
tuserkay 0.8877871632575989
tusermelissa 0.804341733455658
tuseral 0.7983123660087585
tusernicky 0.7970232963562012
tusercat 0.790561854839325
tuserdi 0.787990927696228
tusergeo 0.7879773378372192
tusereve 0.7808942794799805
tuserabbie 0.7787960767745972
tusershay 0.7774391174316406

seed: not touhou
touhou 0.858487606048584
touhou project 0.7077557444572449
tokusatsu 0.6249366998672485
tokrev 0.5637601017951965
touya todoroki 0.5554528832435608
toa 0.53392094373703
todoroki touya 0.5284604430198669
toh king 0.5228137969970703
hitori gotou 0.520067572593689

seed: rusame exchange

seed: meat painting
painting 0.6750514507293701
meat 0.6640787720680237
animal art 0.6308740377426147
paint 0.6155071258544922
paintings 0.6053147315979004
oil painting 0.5972870588302612
painter 0.5898380875587463
digital painting 0.5874813795089722
animal illustration 0.5513209104537964
food photography 0.5461929440498352

seed: i love this music video
music video 0.706646203994751
funny vid

In [62]:
from gensim.parsing.preprocessing import *

def strip_newlines(x):
    return x.replace('\\n', ' ')

filters = [
    strip_newlines,
    strip_tags,
    strip_punctuation,
    strip_multiple_whitespaces,
    remove_stopwords,
]

df = post_df.sample(100)
df['bodyp'] = df.body.map(lambda x: ' '.join(preprocess_string(x, filters)))
df['titlep'] = df.title.map(lambda x: preprocess_string(x, filters))
df['doc'] = list(map(lambda x: ' '.join(x[0] + x[1]), zip(df.titlep, df.root_tags)))
df[['blog_url', 'doc', 'tags']]

,blog_url,doc,tags
100535,indecisivecaffeineaddict.tumblr.com,Me mutuals logging Tumblr daily harvest hoes b...,[lol]
45594,magnvmchasma.tumblr.com,meredithhunnter blog Gustav Heurlin aka Gustav...,"[holy fire, cash]"
30775,small-giant-plans.tumblr.com,books sense ve read tbh nona the ninth gideon ...,"[locked tomb, nona the ninth spoilers]"
68474,colourthewavesandwalls.tumblr.com,fluffygif Magical moment captured nature wild ...,"[animals, nature, you queue here?]"
64511,traditionalfly.tumblr.com,Gossips Vadim Gorodnitsky birds nature photogr...,"[photography, birds, animals]"
...,...,...,...
66563,dabbingonronpa.tumblr.com,One Prime transformers prime tfp transformers ...,"[JUNE DARBY XD XD, go QUEEN]"
5406,umidays.tumblr.com,soft babies 🤍 they’re everything to me 🥺 akian...,"[chainsaw man, fanart]"
82956,simstoricalishccfinds.tumblr.com,Simblreen 2020 Treat 1The Sophia 1824 Evening ...,[ts4 regency]
6452,karmahope.tumblr.com,lowkey haikyuu phase haikyuu haikyuu textposts...,[hq]


In [6]:

documents = []
docs_by_post = {}
tags_by_post = {}
for i, row in df.iterrows():
    documents.append((i, row['doc'][:100], None))
    tags_by_post[i] = row['tags']
    docs_by_post[i] = row['doc']

In [ ]:
batch_size = 1000
for i in tqdm(range(0, len(documents), batch_size)):
    embeddings.upsert(documents[i:i+batch_size])

 50%|█████████████████████████████████▋                                  | 52/105 [30:50<30:44, 34.81s/it]

In [211]:
def printdf(post_df, tag):
    rows = []
    for doc, score in embeddings.search(tag, 25):
        rows.append({
            "tag": documents[doc],
            "score": score,
        })
    display(pd.DataFrame(rows))

for i, row in post_df.sample(1).iterrows():
    for tag in row['tags']:
        print(tag)
        printdf(post_df, tag)

Srvlst


,tag,score
0,"(67308, Srvlst, None)",1.000000
1,"(37643, Srvlst, None)",1.000000
2,"(102622, sr, None)",0.666234
3,"(82331, sr, None)",0.666234
4,"(77985, sr, None)",0.666234
5,"(18129, sr, None)",0.666234
6,"(13605, sr, None)",0.666234
7,"(13403, sr, None)",0.666234
8,"(46578, sr, None)",0.666234
9,"(43145, sr, None)",0.666234


In [104]:
def printdf(post_df, tag):
    rows = []
    for doc, score in embeddings.search(tag, 5):
        rows.append({
            "doc": docs_by_post[doc],
            "tag": tags_by_post[doc],
            "score": score,
        })
    display(pd.DataFrame(rows))

for i, row in post_df.sample(1).iterrows():
    for tag in row['tags']:
        print(tag)
        printdf(post_df, tag)

video games


,doc,tag,score
0,Youth fascination technology They love phone g...,[q],0.488692
1,boozerman DRAGON AGE INQUISITION motionresque ...,[dragon age],0.465870
2,SILT ∙ Spiral Circus 2022 silt gamingedit indi...,"[rbs, games, silt]",0.441177
3,slushiecafe bark bark plays owl city idog nost...,"[idog, dogs, robots]",0.436060
4,bamf Cyberpunk 2077 ↪ RIDERS ON THE STORM cp20...,"[wife, cyberpunk 2077, panam palmer, panam my ...",0.413248


genshin impact


,doc,tag,score
0,recent genshin impact tighnari genshin tighnari,"[shroom tigh, tighnari]",0.738725
1,My bro got traumatized genshin childe genshin ...,"[reblogs, text, Queue will have order]",0.648070
2,HAPPY BIRTHDAY TO YOU Wanderer Scaramouche Gen...,"[other people's art, genshin impact, scaramouc...",0.535647
3,nation divided genshin impact genshin genshin ...,"[genshin impact, raiden ei, art]",0.518707
4,cfeather light sea my art genshin impact xiao ...,"[never played genshin but this is gorgeous, ar...",0.500026


alhaitham


,doc,tag,score
0,Congratulations Al Haitham relatable character...,[genshin impact],0.532969
1,I’m process writing “Alhaitham propaganda” pos...,"[alhaitham, genshin impact, breaking news: the...",0.494797
2,This I imagine Alhaitham messes Kaveh Addition...,"[alhaitham, kaveh, art, genshin, kavetham]",0.485261
3,اسلام آباد ڈسٹرکٹ بار ایسوسی ایشن کے انتخابی ن...,"[breaking news, pakistan, latest news in pakis...",0.444291
4,Al Haitham The Fattening Knowledge belly kink ...,[inflation art],0.416389


In [107]:
len(documents)

104246